<a href="https://colab.research.google.com/github/pela-andrea/people-analytics-case/blob/feat%2Fdivisao-tabelas/scripts/02_divisao_tabelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estudo de Base de Dados RH – Case Técnico

#Introdução

Neste notebook, daremos continuidade ao processo de preparação dos dados para análise, realizando a classificação e segmentação da base original em múltiplas tabelas organizadas segundo critérios específicos. O objetivo é estruturar os dados de forma mais eficiente para análises futuras e uso no Power BI, aplicando conceitos de modelagem como fato e dimensão, quando pertinente.

As principais etapas deste notebook incluem:

- Aplicar classificações relevantes para separar as informações em grupos lógicos;
- Dividir a base de dados original em diversas tabelas e nomear os arquivos gerados seguindo o padrão **tabela_nome.csv**, garantindo organizaçãoo;
- Salvar as tabelas finais para utilização no PowerBI.


#**Etapa 01 - Exploração dos dados**

##Declaração das LIBS

In [24]:
# Principais
import pandas as pd
import numpy as np

# Para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Para tratamento de datas e warnings
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#Para salvar os arquivos gerados, localmente
from google.colab import files

##Configuração do pandas

In [25]:
# Exibir todas as colunas do dataframe
pd.set_option('display.max_columns', None)

##Importação e Extração dos dados

In [26]:
#Importando a base original diretamente do repositório GitHub
url = 'https://raw.githubusercontent.com/pela-andrea/people-analytics-case/main/data/base_corrigida.csv'

df = pd.read_csv(url, sep=';', decimal=',', encoding='utf-8')

In [27]:
#Volume do DataSet
df.shape

(50000, 34)

In [28]:
df.head()

,ID do Colaborador,Cargo,Departamento,Localização,Salário,Data de Admissão,Data de Demissão,Está Ativo,Dias de Falta,Dias de Licença Médica,Dias de Férias Usufruídos,Banco de Horas,Horas Extras,Quantidade de Atrasos,Gênero,Estado Civil,Número de Dependentes,Nível de Escolaridade,Avaliação de Desempenho,Percentual de Bônus,Turno,Tipo de Contrato,Centro de Custo,Status de Conformidade,Plano de Saúde,Email,Tempo de Empresa (anos),Período de Experiência Concluído,ID do Gestor,Data da Última Promoção,Data do Último Treinamento,Flag: Promoção após Demissão,Flag: Demissão antes de Admissão,Flag: Treinamento Antes da Admissão
0,EMP_00001,Gerente de RH,Financeiro,Salvador,6993.43,2022-02-26,NaN,Sim,3,2,13,52,20,6,Outro,Viúvo(a),4,Graduação,5,7.58,Manhã,Permanente,CC013,Em conformidade,Padrão,emp_00001@company.com,3.34,Sim,EMP_00495,NaN,2024-07-04,Não,Não,Não
1,EMP_00002,Engenheiro de Dados,Marketing,Belo Horizonte,5723.47,2023-03-04,NaN,Sim,1,1,14,-80,86,6,Masculino,Casado(a),1,Graduação,5,8.72,Manhã,Temporário,CC026,Em conformidade,Básico,emp_00002@company.com,2.33,Sim,EMP_00345,2024-07-21,2024-02-01,Não,Não,Não
2,EMP_00003,Desenvolvedor Python,Operações,Salvador,7295.38,2010-09-01,NaN,Sim,3,5,25,149,23,1,Feminino,Divorciado(a),1,Mestrado,4,7.93,Manhã,Permanente,CC010,Em conformidade,Básico,emp_00003@company.com,14.84,Sim,EMP_00306,2014-10-18,2024-01-11,Não,Não,Não
3,EMP_00004,Analista Financeiro,Operações,Porto Alegre,9046.06,2020-02-26,NaN,Sim,2,1,21,-92,22,5,Masculino,Casado(a),0,Graduação,3,5.27,Manhã,Permanente,CC008,Em conformidade,Padrão,emp_00004@company.com,5.35,Sim,EMP_00971,NaN,2025-01-05,Não,Não,Não
4,EMP_00005,Engenheiro de Dados,Recursos Humanos,Curitiba,5531.69,2011-06-22,NaN,Sim,1,2,15,93,72,3,Masculino,Solteiro(a),0,Graduação,2,10.95,Tarde,Permanente,CC013,Em conformidade,Premium,emp_00005@company.com,14.03,Sim,EMP_00024,NaN,2025-01-19,Não,Não,Não


In [30]:
colunas = df.columns.tolist()
print(colunas)

['ID do Colaborador', 'Cargo', 'Departamento', 'Localização', 'Salário', 'Data de Admissão', 'Data de Demissão', 'Está Ativo', 'Dias de Falta', 'Dias de Licença Médica', 'Dias de Férias Usufruídos', 'Banco de Horas', 'Horas Extras', 'Quantidade de Atrasos', 'Gênero', 'Estado Civil', 'Número de Dependentes', 'Nível de Escolaridade', 'Avaliação de Desempenho', 'Percentual de Bônus', 'Turno', 'Tipo de Contrato', 'Centro de Custo', 'Status de Conformidade', 'Plano de Saúde', 'Email', 'Tempo de Empresa (anos)', 'Período de Experiência Concluído', 'ID do Gestor', 'Data da Última Promoção', 'Data do Último Treinamento', 'Flag: Promoção após Demissão', 'Flag: Demissão antes de Admissão', 'Flag: Treinamento Antes da Admissão']


Serão necessárias algumas alterações e tratamentos na tabela original

#Divisão da tabela em fato/dimensão
Por que usar fato/dimensão aqui?
A base simula um sistema de RH, com métricas e atributos de colaboradores.

- Temos várias colunas descritivas (boas para dimensões).

- Temos medidas numéricas e de tempo (ótimas para a tabela fato).

- Isso também facilita filtros e relacionamentos no Power BI.

##Organização das tabelas

tabela_d_colaborador
Informações pessoais e contratuais

In [ ]:
tabeça_dColaborador = df[[
    "ID do Funcionário", "Cargo", "Departamento", "Localização", "Data de Admissão",  "Está Ativo",
    "Gênero", "Estado Civil", "Nº de Dependentes", "Nível de Escolaridade", "Tipo de Contrato",
    "Centro de Custo", "Plano de Saúde", "Email", "ID do Gestor", "Data de Desligamento",
    "Tempo de Casa (anos)", "Turno"
]]


tabela_d_remuneracao
Informações de salário, bonus, tempo de empresa

In [ ]:
dim_remuneracao = df[[
    "ID do Funcionário", "Salário", "Bônus (%)", "Tempo de Empresa (anos)",
    "Período de Experiência Concluído", "Plano de Saúde", "Compliance"
]]


tabela_f_jornada Horas, ausências e atrasos

In [ ]:
fato_jornada = df[[
    "ID do Funcionário", "Dias de Falta", "Dias de Licença Médica", "Dias de Férias",
    "Horas no Banco", "Horas Extras", "Ocorrências de Atraso"
]]


tabela_f_desempenho   Performance, promoções e treinamentos

In [ ]:
fato_desempenho = df[[
    "ID do Funcionário", "Nota de Performance", "Data da Última Promoção",
    "Data do Último Treinamento"
]]


In [ ]:
dim_colaborador.to_csv("tabela_dim_colaborador.csv", index=False)
dim_remuneracao.to_csv("tabela_dim_remuneracao.csv", index=False)
fato_jornada.to_csv("tabela_fato_jornada.csv", index=False)
fato_desempenho.to_csv("tabela_fato_desempenho.csv", index=False)


In [ ]:
from google.colab import files
files.download("tabela_dim_colaborador.csv")
files.download("tabela_dim_remuneracao.csv")
files.download("tabela_fato_jornada.csv")
files.download("tabela_fato_desempenho.csv")
